In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head() 

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,kalmunai,LK,78.01,heavy intensity rain,7.4167,81.8167,VS Villa
1,5,avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
2,6,salinopolis,BR,78.37,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
3,9,elat,IL,77.00,clear sky,29.5581,34.9482,Club Hotel Eilat
4,10,butaritari,KI,82.47,few clouds,3.0707,172.7902,Isles Sunset Lodge


In [43]:
my_vacation = vacation_df[vacation_df["Country"]=="MX"]
my_vacation

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
42,135,cabo san lucas,MX,86.00,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
108,340,bucerias,MX,75.20,clear sky,20.7500,-105.3333,Villa del Palmar Flamingos Beach Resort & Spa ...
110,349,tecoanapa,MX,84.45,clear sky,16.5167,-98.7500,Hotel Manglares
111,350,acapulco,MX,86.00,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
149,474,lazaro cardenas,MX,84.76,clear sky,17.9583,-102.2000,Hotel Quinta Antigua
183,646,puerto escondido,MX,84.20,few clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa


In [21]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [22]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = my_vacation.loc[my_vacation["City"]=="cabo san lucas"]
vacation_end = my_vacation.loc[my_vacation["City"]=="cabo san lucas"]
vacation_stop1 = my_vacation.loc[my_vacation["City"]=="bucerias"]
vacation_stop2 = my_vacation.loc[my_vacation["City"]=="lazaro cardenas"] 
vacation_stop3 = my_vacation.loc[my_vacation["City"]=="acapulco"] 

In [48]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.iloc[0,5:7].to_numpy()
end = vacation_end.iloc[0,5:7].to_numpy()
stop1 = vacation_stop1.iloc[0,5:7].to_numpy()
stop2 = vacation_stop2.iloc[0,5:7].to_numpy()
stop3 = vacation_stop3.iloc[0,5:7].to_numpy
stop3
#stop1
#stop2
#stop3

<bound method IndexOpsMixin.to_numpy of Lat    16.8634
Lng   -99.8901
Name: 111, dtype: object>

In [49]:
fig1 = gmaps.figure()
cabo = (22.8909, -109.9124)
bucerias = (20.75, -105.3333)
lazaro = (17.9583, -102.2)
acapulco = (16.8634, -99.8901)
fig1 = gmaps.figure()

In [53]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
trip = gmaps.directions_layer(cabo, cabo, waypoints=[bucerias, lazaro, acapulco],
                             travel_mode='BICYCLING')
fig1.add_layer(trip)
fig1

Figure(layout=FigureLayout(height='420px'))

In [54]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,135,cabo san lucas,MX,86.00,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
1,340,bucerias,MX,75.20,clear sky,20.7500,-105.3333,Villa del Palmar Flamingos Beach Resort & Spa ...
2,474,lazaro cardenas,MX,84.76,clear sky,17.9583,-102.2000,Hotel Quinta Antigua
3,350,acapulco,MX,86.00,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco


In [55]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in itinerary_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        itinerary_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

In [57]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map.
fig2 = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_hotel_info)

fig2.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))